## import

In [3]:
import os
import glob
root_path= '/home/heiscold/korean_whisper_finetune/hf/'
os.chdir(root_path)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import librosa
import IPython.display as ipd
import soundfile as sf
from tqdm.auto import tqdm

import torch
import torch.nn as nn
import torchaudio

# from conformer import Conformer
os.environ["CUDA_VISIBLE_DEVICES"]= "0"

/home/heiscold/miniconda3/envs/elu/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 1. 오디오 파일 경로 취합

[Reference](https://colab.research.google.com/drive/1wSp66cLd0C6WzR9hCdvlHfIEjcd2ZfEj?usp=sharing#scrollTo=izh5If2Xq875)

In [1]:
text_base_dir = f"/home/heiscold/korean_whisper_finetune/data/valid/D02_Transcription/"
audio_base_dir = f"/home/heiscold/korean_whisper_finetune/data/valid/D02_Audio/"
save_dir = f"/home/heiscold/korean_whisper_finetune/data/"

In [4]:
# 1. 오디오 파일 경로 취합
# import gliob

path = audio_base_dir + "*/*/*" # "오디오 파일들이 포함된 경로를 입력한다. - 예) /content/drive/MyDrive/NLP_Project_data/raw_data/*"
raw_data_list = glob.glob(path)
raw_data_list = sorted(raw_data_list)

In [5]:
print(f"file_list : {raw_data_list[:10]}")
print(len(raw_data_list))

file_list : ['/home/heiscold/korean_whisper_finetune/data/valid/D02_Audio/J05/S007963/0001.wav', '/home/heiscold/korean_whisper_finetune/data/valid/D02_Audio/J05/S007963/0002.wav', '/home/heiscold/korean_whisper_finetune/data/valid/D02_Audio/J05/S007963/0003.wav', '/home/heiscold/korean_whisper_finetune/data/valid/D02_Audio/J05/S007963/0004.wav', '/home/heiscold/korean_whisper_finetune/data/valid/D02_Audio/J05/S007963/0005.wav', '/home/heiscold/korean_whisper_finetune/data/valid/D02_Audio/J05/S007963/0006.wav', '/home/heiscold/korean_whisper_finetune/data/valid/D02_Audio/J05/S007963/0007.wav', '/home/heiscold/korean_whisper_finetune/data/valid/D02_Audio/J05/S007963/0008.wav', '/home/heiscold/korean_whisper_finetune/data/valid/D02_Audio/J05/S007963/0009.wav', '/home/heiscold/korean_whisper_finetune/data/valid/D02_Audio/J05/S007963/0010.wav']
15211


## 2. 텍스트 파일 경로 취합

In [6]:
# 2. 텍스트 파일 경로 취합
path = text_base_dir + "*/*/*" # "텍스트 파일들이 포함된 경로를 입력한다. - 예) /content/drive/MyDrive/NLP_Project_data/labeled_data/*"
labeled_data_list = glob.glob(path)
# 레이블 데이터에는 json 데이터가 폴더별로 하나씩 있으므로 txt 파일만을 골라낸다.
labeled_data_list = sorted([file for file in labeled_data_list if file.endswith(".txt")])

In [7]:
print(f"file_list : {labeled_data_list[:10]}")
print(len(labeled_data_list))

file_list : ['/home/heiscold/korean_whisper_finetune/data/valid/D02_Transcription/J05/S007963/0001.txt', '/home/heiscold/korean_whisper_finetune/data/valid/D02_Transcription/J05/S007963/0002.txt', '/home/heiscold/korean_whisper_finetune/data/valid/D02_Transcription/J05/S007963/0003.txt', '/home/heiscold/korean_whisper_finetune/data/valid/D02_Transcription/J05/S007963/0004.txt', '/home/heiscold/korean_whisper_finetune/data/valid/D02_Transcription/J05/S007963/0005.txt', '/home/heiscold/korean_whisper_finetune/data/valid/D02_Transcription/J05/S007963/0006.txt', '/home/heiscold/korean_whisper_finetune/data/valid/D02_Transcription/J05/S007963/0007.txt', '/home/heiscold/korean_whisper_finetune/data/valid/D02_Transcription/J05/S007963/0008.txt', '/home/heiscold/korean_whisper_finetune/data/valid/D02_Transcription/J05/S007963/0009.txt', '/home/heiscold/korean_whisper_finetune/data/valid/D02_Transcription/J05/S007963/0010.txt']
15211


### 텍스트 데이터는 파일의 내용을 추출하여 데이터프레임으로 만든다.

In [8]:
transcript_list = []
for labeled_data in tqdm(labeled_data_list):
    with open(labeled_data, 'r', encoding='UTF8') as f:
        line = f.readline()
        transcript_list.append(line)

df = pd.DataFrame(data=transcript_list, columns = ["transcript"])

100%|██████████| 15211/15211 [00:00<00:00, 57840.69it/s]


In [9]:
# 텍스트 데이터로 만든 데이터프레임에 음성 파일 경로 컬럼을 추가
df["raw_data"] = raw_data_list

In [10]:
# Null data 유무 확인
df.isnull().values.sum()

0

In [11]:
print(df.shape)
df.head()

(15211, 2)


,transcript,raw_data
0,네 일자리창설과입니다.,/home/heiscold/korean_whisper_finetune/data/va...
1,네 그 희망n/ 근로 사업하고 공공 근로 사업에 대한 민원 제기 건이 접수 되어서 ...,/home/heiscold/korean_whisper_finetune/data/va...
2,o/ n/ 네 그 접수는 된 것 같은데 b/ 뭐 (1)/(한) 사람이 뭐 몇 년씩 ...,/home/heiscold/korean_whisper_finetune/data/va...
3,n/ 이분 말씀으로는 그 몇 년째 계속 공공 근로나 희망 근로하시는 분께서 b/ 뭐...,/home/heiscold/korean_whisper_finetune/data/va...
4,n/ 어 이런 걸 좀 잘 조치를 취해 달라는 내용으로 신고해 주셨습니다.,/home/heiscold/korean_whisper_finetune/data/va...


In [12]:
df.tail()

,transcript,raw_data
15206,네 번호가 지역번호,/home/heiscold/korean_whisper_finetune/data/va...
15207,예 알겠습니다.,/home/heiscold/korean_whisper_finetune/data/va...
15208,궁금하신 점 있으신가요?,/home/heiscold/korean_whisper_finetune/data/va...
15209,아니요 없어요.,/home/heiscold/korean_whisper_finetune/data/va...
15210,잘 처리 되시길 바랍니다.,/home/heiscold/korean_whisper_finetune/data/va...


## Check!

In [13]:
idx = 2222

sentence = df.loc[idx, "transcript"]
audio_path = df.loc[idx, "raw_data"]

print(f"transcript : {sentence}")
print(f"audio_path : {audio_path}")

transcript : 네 그 인감이 있으시면
audio_path : /home/heiscold/korean_whisper_finetune/data/valid/D02_Audio/J06/S004097/0038.wav


In [14]:
# librosa load
wav, sr = librosa.load(audio_path, sr = None)

# Shape
print(f"wav.shape: {wav.shape}")
# 샘플링레이트
print(f"sampling_rate: {sr}")

# Original Mixed
ipd.display(ipd.Audio(wav, rate =sr))

wav.shape: (32000,)
sampling_rate: 8000


In [15]:
idx = 1111

sentence = df.loc[idx, "transcript"]
audio_path = df.loc[idx, "raw_data"]

print(f"transcript : {sentence}")
print(f"audio_path : {audio_path}")

transcript : n/ o/ 예 (밥) 밥 먹는 거는 뭐 그런 데로 밥맛은 없어도 때는 걸르지 않고 있는데.
audio_path : /home/heiscold/korean_whisper_finetune/data/valid/D02_Audio/J05/S008009/0015.wav


In [16]:
# librosa load
wav, sr = librosa.load(audio_path, sr = None)

# Shape
print(f"wav.shape: {wav.shape}")
# 샘플링레이트
print(f"sampling_rate: {sr}")

# Original Mixed
ipd.display(ipd.Audio(wav, rate =sr))

wav.shape: (77176,)
sampling_rate: 8000


## 3. 저장

In [21]:
save_dir

'/home/heiscold/korean_whisper_finetune/data/'

In [17]:
# 필요에 따라 데이터프레임을 csv 파일로 저장한다.
df.to_csv(save_dir + "path_and_transcript.csv", index= False)

In [18]:
# csv 파일을 로드하여 df로 변환
df = pd.read_csv(save_dir + "path_and_transcript.csv")

In [19]:
print(df.shape)
df.head()

(15211, 2)


,transcript,raw_data
0,네 일자리창설과입니다.,/home/heiscold/korean_whisper_finetune/data/va...
1,네 그 희망n/ 근로 사업하고 공공 근로 사업에 대한 민원 제기 건이 접수 되어서 ...,/home/heiscold/korean_whisper_finetune/data/va...
2,o/ n/ 네 그 접수는 된 것 같은데 b/ 뭐 (1)/(한) 사람이 뭐 몇 년씩 ...,/home/heiscold/korean_whisper_finetune/data/va...
3,n/ 이분 말씀으로는 그 몇 년째 계속 공공 근로나 희망 근로하시는 분께서 b/ 뭐...,/home/heiscold/korean_whisper_finetune/data/va...
4,n/ 어 이런 걸 좀 잘 조치를 취해 달라는 내용으로 신고해 주셨습니다.,/home/heiscold/korean_whisper_finetune/data/va...


In [20]:
df.head()

,transcript,raw_data
0,네 일자리창설과입니다.,/home/heiscold/korean_whisper_finetune/data/va...
1,네 그 희망n/ 근로 사업하고 공공 근로 사업에 대한 민원 제기 건이 접수 되어서 ...,/home/heiscold/korean_whisper_finetune/data/va...
2,o/ n/ 네 그 접수는 된 것 같은데 b/ 뭐 (1)/(한) 사람이 뭐 몇 년씩 ...,/home/heiscold/korean_whisper_finetune/data/va...
3,n/ 이분 말씀으로는 그 몇 년째 계속 공공 근로나 희망 근로하시는 분께서 b/ 뭐...,/home/heiscold/korean_whisper_finetune/data/va...
4,n/ 어 이런 걸 좀 잘 조치를 취해 달라는 내용으로 신고해 주셨습니다.,/home/heiscold/korean_whisper_finetune/data/va...
